In [1]:
from tensorflow import keras
import tensorflow as tf
from keras.layers import Conv2D,Flatten,Dense,MaxPooling2D,Dropout


2023-07-25 22:07:09.909406: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-25 22:07:09.956640: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-25 22:07:10.219218: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-25 22:07:10.221053: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 22:07:11.939877: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator()

In [3]:
train = datagen.flow_from_directory('New Plant Diseases Dataset(Augmented)/train',class_mode='categorical',batch_size=32,target_size=(256,256))
test = datagen.flow_from_directory('test',class_mode='categorical',batch_size=32,target_size=(256,256))
valid = datagen.flow_from_directory('New Plant Diseases Dataset(Augmented)/valid',class_mode='categorical',batch_size=32,target_size=(256,256))

Found 70295 images belonging to 38 classes.
Found 33 images belonging to 1 classes.
Found 17572 images belonging to 38 classes.


In [4]:
x_train,xt=train.next()
print('Batch shape=',x_train.shape,'min=',x_train.min(),'max=',x_train.max())

Batch shape= (32, 256, 256, 3) min= 0.0 max= 255.0


In [5]:
class_names=['Apple_scab','Apple_Black_rot','Cedar_apple_rust','healthy_apple',
'healthy_blueberry','healthy_cherry','cherry_Powdery_mildew','corn_cercospora_leaf_spot gray_leaf_spot',
'corn_common_rust','healthy_corn','corn_northern_leaf_blight','grape_black_rot',
'grapes_Esca(black_measles)','healthy_grape','grape_leaf_blight(isariopsis_leaf_spot)','Orange_haunglongbing(citrus_greening)',
'peach_bacterial_spot','peach_healthy','pepper_bell_bacterial_spot','healthy_pepper_bell','potato_early_blight',
'healthy_potato','potato_late_blight','healthy_raspberry','soybean_healthy','squash_powdery_mildew',
'strawberry_healthy','strawberry_leaf_scrorch','tomato_bacterial_spot','tomato_early_blight','tomato_healthy',
'tomato_late_blight','tomato_leaf_mold','tomato_septoria_leaf_spot','tomato_spider_mites two_spotted_spider_mite',
'tomato_target_spot','tomato_mosaic_virus','tomato_yellow_leaf_curl_virus'
]

In [6]:
model=keras.models.Sequential([
    #1
    keras.layers.Conv2D(filters=90,kernel_size=(5,5),strides=(2,2),activation='relu',input_shape=(256,256,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    #2
    keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3)),
    #3
    keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    #4
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    #5
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    
    keras.layers.Flatten(),
    #6
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    #7
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    #8
    keras.layers.Dense(38,activation='softmax')

])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 90)      6840      
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 90)     360       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 90)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 63, 63, 256)       207616    
                                                                 
 batch_normalization_1 (Batc  (None, 63, 63, 256)      1024      
 hNormalization)                                                 
                                                        

In [7]:
len(train)

2197

In [8]:
#trained with only one epoch beacause of gpu problem
history=model.fit(train,epochs=10,validation_data=valid)

Epoch 1/10


2023-07-25 22:10:48.266473: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


   8/2197 [..............................] - ETA: 1:53:21 - loss: 18.4218 - accuracy: 0.0508

In [ ]:
model.save('model.h5')

In [9]:
import pickle
pickle.dump(model,open('plant_model.pkl','wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......batch_normalization
.........vars
............0
............1
............2
............3
......batch_normalization_1
.........vars
............0
............1
............2
............3
......batch_normalization_2
.........vars
............0
............1
............2
............3
......batch_normalization_3
.........vars
............0
............1
............2
............3
......batch_normalization_4
.........vars
............0
............1
............2
............3
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......conv2d_2
.........vars
............0
............1
......conv2d_3
.........vars
............0
............1
......conv2d_4
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
.